In [1]:
from typing import List
from itertools import accumulate
from time import perf_counter_ns
import numpy as np
from scipy.sparse import csc_matrix, kron, identity
import cirq
import openfermion as of
from openfermionpyscf import run_pyscf
import quimb.tensor as qtn
from quimb.tensor.tensor_1d import MatrixProductState, MatrixProductOperator
from convert import to_groups_of
from error_pert import get_v2_sarray
from qpe_trotter import v2_pauli_sum, v2_qubop
from kcommute import get_si_sets
from tensor_network_common import pauli_sum_to_mpo, mps_to_vector

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

This notebook requires quantum-toolbox installed: https://github.com/jdrowland/quantum-toolbox/

The following timing and validation results are run on 10 amd20 nodes with 16 GB of memory.  Comparison is made to cirq.PauliSum only.  Timing results are significantly worse on this hardware than the original notebook timing results.  This likely owes to superior memory streaming on the original (Mac Pro?) hardware.  Speedups achieved are plausibly robust provided computations remain memory-bound.  From other work I have done, AMD24 hardware remains the best option on ICER for memory bound compute, and should be constrained for cluster work of this nature.  Parallelization is implemented to demonstrate that the remaining dominant component of the computation is in the large number of expectation values which are embarassingly parallel.  Highly recommend parallelizing this differently than demonstrated in this notebook in practice by utilizing the cluster's scavenger queue to produce a very large number of cores as you scale beyond LiH.  Such an approach is approach is not friendly to the notebook format however, and as such we utilize multiprocessing.

The quantum-toolbox contains a relatively diverse set of modular functions built upon symplectic data structures primarily intended for grouping Pauli operators and computing their expectation values.  Signficant performance detriments result from changing between formats and as such it is recommended to work exclusively with the symplectic form as much as possible.  Ideally all conversion happen directly to and from symplectic form without intermediate products.  This effect shouldn't inherently improve scaling, but it is likely for memory bound processes such as commutativity checks, the cirq objects induce non-linear overhead.  

The toolbox contains methods for converting between and computing with different formats such as cirq, qiskit, and tensor networks.

To highlight a couple of speedups:  Grouping occurs in a mere 15 ms as compared to the original 5.9s, a 393x speedup.  This speedup grows as problems become more challenging.  Serial computation of expectation values exhibits a 2.26x speedup, while the parallel version is an additional 6.5x on the expectation value computation, not quite achieving the theoretical 10x, for a relative speedup (including build time) of 4.2x.  No comparison is made to the original because for a fair comparison one would also need to parallelize the original cirq based code as well.  

In general, with enough cores this computation can be driven toward the build time + grouping time.  Parallelization of the symplectic build_v2_terms is also possible should this become a bottleneck, though a little bit trickier to do.  A reasonable approach might be to produce the O(N^2) list of commutators [A,B], and then proceed to parallelize the last computation.  Combining the results of the different computations results in some overhead, as an 'abstract' sum of strings need to be combined, rather than scalars.  This can be avoided by parallelizing the expectation values at the level of the commutator also.

The embarassingly parallel aspect of the expectation values still remains a non-trivial problem as you scale to a problem of the size of OWP.  The number of expectation values scales very poorly with the number of terms, something like O(M^3) where M is the number of terms, so scaling to OWP is at worst a trillion times worse.  Even with scavenger queue, the number of cores you could access in parallel are roughly 10^4, so this remains prohibitive.  Perhaps computing all of the commutators is possible, but computing their expectation values *exactly* for a general state seems unlikely.  For grouping, the number of expectation values scales linearly with M, so this remains feasible to do.  Perhaps you could still do this exactly for a relatively unentangled state, such as HF, or by somehow truncating the number of expectation values required, though this seems unlikely to meet your goals.  


At the LiH problem size, I think you will be always worse off doing it this way rather than sparse matrix methods, but as problem size grows, I expect the sparse methods to eventually be worse, perhaps comparable for water with maximal parallelism.

In [2]:
molec = "LiH"
basis = "sto-3g"
n_elec = 4
geometry = of.chem.geometry_from_pubchem(molec)
multiplicity = 1
molecule = of.chem.MolecularData(
    geometry, basis, multiplicity
)
molecule = run_pyscf(molecule, run_scf=1, run_fci=1)
print(f"HF energy:", molecule.hf_energy)
print(f"FCI energy:", molecule.fci_energy)
hamiltonian = molecule.get_molecular_hamiltonian()
hamiltonian_qubop = of.transforms.jordan_wigner(hamiltonian)
hamiltonian_psum = of.transforms.qubit_operator_to_pauli_sum(hamiltonian_qubop)

nq = of.utils.count_qubits(hamiltonian_qubop)
nterms = len(hamiltonian_qubop.terms)
print(f"Hamiltonian has {nq} qubits and {nterms} terms.")

HF energy: -7.76736213574856
FCI energy: -7.784460280031227
Hamiltonian has 12 qubits and 631 terms.


In [3]:
qs = cirq.LineQubit.range(nq)
hamiltonian_mpo = pauli_sum_to_mpo(hamiltonian_psum, qs, 100)
dmrg = qtn.DMRG(hamiltonian_mpo, bond_dims=15)
converged = dmrg.solve()
if not converged:
    print("DMRG did not converge.")
ground_state = dmrg.state
ground_state_vec = mps_to_vector(ground_state)

sorted_inds = ['k0', 'k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11']


In [4]:
start_time = perf_counter_ns()
groups = get_si_sets(hamiltonian_psum, nq)
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"There are {len(groups)} groups.")
print(f"Elapsed time: {elapsed_time:4.5e} ns")

There are 42 groups.
Elapsed time: 5.92668e+09 ns


## Sparse matrices

In [10]:
# Use the code from the paper.
start_time = perf_counter_ns()
group_qubops = to_groups_of(groups)
sparse_frag_ops = []
# Convert group operators to sparse matrices.
# We must make sure the matrices have all the same size.
# If a matrix is not big enough, tensor it with I on the right.
for op in group_qubops:
    nq_op = of.utils.count_qubits(op)
    op_sparse = of.linalg.get_sparse_operator(op)
    if nq_op != nq:
        eye_diff = identity(2 ** (nq - nq_op), dtype="complex", format='csc')
        new_op = kron(op_sparse, eye_diff, format="csc")
        sparse_frag_ops.append(new_op)
    else:
        sparse_frag_ops.append(op_sparse)
v2_sparse = get_v2_sarray(sparse_frag_ops)
eps2 = np.vdot(ground_state_vec, v2_sparse @ ground_state_vec).real
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2} in {elapsed_time:4.5e} ns")

Got eps2=0.009780710988690324 in 3.70797e+09 ns


## `cirq.PauliSum`

In [6]:
start_time = perf_counter_ns()
group_psums = [sum(group) for group in groups]
v2_psum = v2_pauli_sum(group_psums)
qubit_map = {q: i for i, q in enumerate(qs)}
eps2_psum = v2_psum.expectation_from_state_vector(ground_state_vec, qubit_map)
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2_psum} in {elapsed_time:4.5e} ns")

Got eps2=(0.00978075160645751-1.2363017708628819e-20j) in 1.84036e+12 ns


## `of.QubitOperator`

In [ ]:
start_time = perf_counter_ns()
group_qubops = to_groups_of(groups)
v2_op = v2_qubop(group_psums)
qubit_map = {q: i for i, q in enumerate(qs)}
eps2_qubop = v2_psum.expectation_from_state_vector(ground_state_vec, qubit_map)
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2_qubop} in {elapsed_time:4.5e} ns")

## quantum-toolbox (symplectic representation)

In [5]:
# Use quantum-toolbox for native symplectic representation
import sys
sys.path.insert(0, '../quantum-toolbox')
from qtoolbox.core.pauli import PauliString
from qtoolbox.core.hamiltonian import Hamiltonian
from qtoolbox.converters.openfermion_bridge import from_openfermion
from qtoolbox.grouping import sorted_insertion_grouping
from itertools import accumulate
from multiprocessing import Pool, cpu_count

# Convert OpenFermion QubitOperator to quantum-toolbox Hamiltonian
n_qubits_qt = of.utils.count_qubits(hamiltonian_qubop)
terms = [from_openfermion(term, coeff, n_qubits_qt)
         for term, coeff in hamiltonian_qubop.terms.items() if term]  # skip identity
ham = Hamiltonian(terms)
print(f"Loaded Hamiltonian: {ham.num_terms()} terms, {ham.num_qubits()} qubits")

# Group using quantum-toolbox's SI
start_group = perf_counter_ns()
group_collection = sorted_insertion_grouping(ham)
time_grouping = perf_counter_ns() - start_group

sym_groups = [list(g.paulis) for g in group_collection.groups]
print(f"Grouped into {len(sym_groups)} groups in {time_grouping/1e9:.3f} s (native symplectic format)")

Loaded Hamiltonian: 630 terms, 12 qubits
Grouped into 42 groups in 0.015 s (native symplectic format)


In [6]:
def fast_commutator_sum(a_terms, b_terms):
    result = []
    for ai in a_terms:
        for bj in b_terms:
            if not ai.commutes_with(bj):  
                p = ai.multiply(bj)       
                p.coeff *= 2              
                result.append(p)
    return result

def apply_pauli_to_state(x_bits, z_bits, n_qubits, state):
    dim = len(state)
    result = state.copy()

    x_bits_of, z_bits_of = 0, 0
    for q in range(n_qubits):
        if x_bits & (1 << q):
            x_bits_of |= (1 << (n_qubits - 1 - q))
        if z_bits & (1 << q):
            z_bits_of |= (1 << (n_qubits - 1 - q))

    if z_bits_of:
        indices = np.arange(dim)
        parity = np.zeros(dim, dtype=np.int8)
        for b in range(n_qubits):
            if z_bits_of & (1 << b):
                parity ^= ((indices >> b) & 1).astype(np.int8)
        result *= (1 - 2*parity)

    if x_bits_of:
        result = result[np.arange(dim) ^ x_bits_of]

    y_bits = x_bits & z_bits
    if y_bits:
        result *= (1j) ** bin(y_bits).count('1')

    return result

def build_v2_terms(sym_groups):
    nterms = len(sym_groups)
    sums_l2r = list(accumulate(sym_groups, lambda a, b: a + b))
    sums_r2l = list(reversed(list(accumulate(reversed(sym_groups), lambda a, b: a + b))))
    sums_r2l.append([])

    v2_terms = []
    for i in range(1, nterms):
        V1 = fast_commutator_sum(sums_l2r[i-1], sym_groups[i])
        for t in fast_commutator_sum(V1, sums_r2l[i+1]):
            t.coeff *= -1/3
            v2_terms.append(t)
        for t in fast_commutator_sum(V1, sym_groups[i]):
            t.coeff *= -1/6
            v2_terms.append(t)
    return v2_terms

def compute_expectation_sequential(v2_terms, psi, n_qubits):
    eps2 = 0.0
    for t in v2_terms:
        p_state = apply_pauli_to_state(t.x_bits, t.z_bits, n_qubits, psi)
        eps2 += (t.coeff * np.vdot(psi, p_state)).real
    return eps2

In [10]:
start_time = perf_counter_ns()
v2_terms = build_v2_terms(sym_groups)
time_build = perf_counter_ns() - start_time

start_exp = perf_counter_ns()
eps2_symplectic = compute_expectation_sequential(v2_terms, ground_state_vec, nq)
time_exp = perf_counter_ns() - start_exp

end_time = perf_counter_ns()
elapsed_ns = end_time - start_time

error = abs(eps2 - eps2_symplectic)
print(f"quantum-toolbox (sequential):")
print(f"   V2 terms:     {len(v2_terms):,}")
print(f"   eps2 =        {eps2_symplectic:.10f}")
print(f"   Build time:   {time_build/1e9:.3f} s")
print(f"   Exp. time:    {time_exp/1e9:.3f} s")
print(f"   Total time:   {elapsed_ns/1e9:.3f} s")
print(f"   Error vs sparse: {error:.2e}")
assert error < 1e-8, f"Results don't match! Error: {error}"
print("   ✓ Validation passed")

quantum-toolbox (sequential):
   V2 terms:     7,513,732
   eps2 =        0.0097807516
   Build time:   83.774 s
   Exp. time:    729.712 s
   Total time:   813.487 s
   Error vs sparse: 8.83e-16
   ✓ Validation passed


In [7]:
def compute_expectation_batch(args):
    terms_data, psi, n_qubits = args
    dim = len(psi)
    total = 0.0
    for x_bits, z_bits, coeff in terms_data:
        result = psi.copy()
        x_bits_of, z_bits_of = 0, 0
        for q in range(n_qubits):
            if x_bits & (1 << q):
                x_bits_of |= (1 << (n_qubits - 1 - q))
            if z_bits & (1 << q):
                z_bits_of |= (1 << (n_qubits - 1 - q))
        if z_bits_of:
            indices = np.arange(dim)
            parity = np.zeros(dim, dtype=np.int8)
            for b in range(n_qubits):
                if z_bits_of & (1 << b):
                    parity ^= ((indices >> b) & 1).astype(np.int8)
            result = result * (1 - 2*parity)
        if x_bits_of:
            result = result[np.arange(dim) ^ x_bits_of]
        y_bits = x_bits & z_bits
        if y_bits:
            result = result * ((1j) ** bin(y_bits).count('1'))
        total += (coeff * np.vdot(psi, result)).real
    return total

def compute_expectation_parallel(v2_terms, psi, n_qubits, n_workers=None):
    if n_workers is None:
        n_workers = max(1, cpu_count())  
    terms_data = [(t.x_bits, t.z_bits, t.coeff) for t in v2_terms]
    
    batch_size = len(terms_data) // n_workers + 1
    batches = [(terms_data[i:i+batch_size], psi, n_qubits) 
               for i in range(0, len(terms_data), batch_size)]
    
    with Pool(n_workers) as pool:
        results = pool.map(compute_expectation_batch, batches)
    return sum(results)

In [11]:
start_time = perf_counter_ns()
v2_terms = build_v2_terms(sym_groups)
time_build = perf_counter_ns() - start_time
print(f"   Build time:   {time_build/1e9:.3f} s")
n_workers = 9
print(f"quantum-toolbox (parallel, {n_workers} workers):")

start_time = perf_counter_ns()
eps2_parallel = compute_expectation_parallel(v2_terms, ground_state_vec, nq, n_workers)
time_parallel = perf_counter_ns() - start_time

error_par = abs(eps2 - eps2_parallel)
print(f"   eps2 =        {eps2_parallel:.10f}")
print(f"   Time:         {time_parallel/1e9:.3f} s")
assert error_par < 1e-8, f"Results don't match! Error: {error_par}"
print("   ✓ Validation passed")

   Build time:   83.538 s
quantum-toolbox (parallel, 9 workers):
   eps2 =        0.0097807110
   Time:         111.085 s
   ✓ Validation passed


## MPO

In [6]:
def to_groups_mpo(groups: List[List[cirq.PauliString]], qs: List[cirq.Qid], max_bond: int) -> List[MatrixProductOperator]:
    """Convert groups from SI into a list of MatrixProductOperators."""

    mpos: List[MatrixProductOperator] = []
    for group in groups:
        group_psum = sum(group)
        group_mpo = pauli_sum_to_mpo(group_psum, qs, max_bond=max_bond)
        mpos.append(group_mpo.copy())
    return mpos

def mpo_add_compress(
    mpo_a: MatrixProductOperator, mpo_b: MatrixProductOperator,
    compress: bool = False, max_bond: int = 100
) -> MatrixProductOperator:
    """Add two MPOs, optionally compressing them."""

    mpo_sum = mpo_a + mpo_b
    if compress:
        mpo_sum.compress(max_bond=max_bond)
    return mpo_sum


def mpo_commutator(mpo_a: MatrixProductOperator, mpo_b: MatrixProductOperator) -> MatrixProductOperator:
    """Take the commutator of two MPOs."""

    return mpo_a.apply(mpo_b) - mpo_b.apply(mpo_a)


def zeros_mpo(nsites: int, phys_dim: int=2) -> MatrixProductOperator:
    """Returns an MPO corresponding to a matirx of all zeros."""

    def fill_fun(shape):
        return np.zeros(shape, dtype=complex)
    
    return MatrixProductOperator.from_fill_fn(fill_fun, L=nsites, bond_dim=1, phys_dim=phys_dim)


def get_v2_contrib_mpo(fragments_list: List[MatrixProductOperator], psi: MatrixProductState, max_bond: int) -> float:
    """Calculate eps2 when the fragments are MPOs."""

    max_mpo_sites = max([len(mpo.tensor_map) for mpo in fragments_list])

    frags_len = len(fragments_list)
    frag_sums_l2r = list(accumulate(fragments_list, lambda a, b: mpo_add_compress(a, b, True, max_bond)))
    temp = reversed(fragments_list)
    frag_sums_r2l = list(accumulate(temp, lambda a, b: mpo_add_compress(a, b, True, max_bond)))
    frag_sums_r2l = list(reversed(frag_sums_r2l))
    frag_sums_r2l.append(zeros_mpo(max_mpo_sites))
    frag_combs_V1_v2 = [(frag_sums_l2r[i-1], fragments_list[i], frag_sums_r2l[i+1]) for i in range (1, frags_len)]
    eps2 = 0
    for i,j,k in frag_combs_V1_v2:
        V1_term = mpo_commutator(i, j)
        term1 = psi.H @ psi.gate_with_mpo(mpo_commutator(V1_term, k))
        term2 = psi.H @ psi.gate_with_mpo(mpo_commutator(V1_term, j))
        eps2 += - term1 *1/3 - term2 *1/6
    return eps2.real

In [13]:
max_mpo_bond = 5
start_time = perf_counter_ns()
mpo_fragments = to_groups_mpo(groups, qs, max_mpo_bond)
end_time = perf_counter_ns()
elapsed_time_convert = end_time - start_time

In [15]:
start_time = perf_counter_ns()
eps2_mpo = get_v2_contrib_mpo(mpo_fragments, ground_state, max_mpo_bond)
end_time = perf_counter_ns()
elapsed_time_calculate = end_time - start_time

print(f"Got eps2={eps2_mpo}.")
print(f"Conversion time: {elapsed_time_convert:4.5e}, calculation time: {elapsed_time_calculate:4.5e}.")
eps2_err = abs(eps2 - eps2_mpo)
print(f"Error {eps2_err:4.5e}")

/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)


Got eps2=0.009230812660028556.
Conversion time: 1.23026e+09, calculation time: 8.41664e+10.
Error 5.40575e-04
